## Example Usage the HecTime Class ##

### Setup ###

In [ ]:
import os, sys

srcdir = os.path.join(os.path.dirname(os.getcwd()), "src")
sys.path.append(srcdir)
from hec.hectime import HecTime  # type: ignore
from hec.interval import Interval  # type: ignore
from IPython.display import display_html

### End-of-month 1Month intervals for a time window ###

In [ ]:
intvl = Interval.getDss("1Month")
start_time = HecTime("2024-01-31, 24:00")
end_time = start_time + "1Y"
count = start_time.computeNumberIntervals(end_time, intvl)
times = [start_time + f"{i}M" for i in range(count)]
for t in times:
    print(t)

### LRTS Daily Times in UTC ###
Note the transition from 13:00 to 12:00 (23 hour day) in the Spring DST transition and then back again (25 hour day) in the Autum DST transistion.

In [ ]:
intvl = Interval.getCwms("1Day")
offset = 7 * Interval.MINUTES["1Hour"]
for month in ("Mar", "Nov"):
    start_time = HecTime(f"01{month}2024, 00:00").atTimeZone("US/Central")
    end_time = start_time + "1M"
    first_time = HecTime(start_time)
    if first_time.adjustToIntervalOffset(intvl, offset) < start_time:
        first_time.increment(1, intvl)
    count = first_time.computeNumberIntervals(end_time, intvl)
    times = [(first_time + f"{i}D").astimezone("UTC") for i in range(count)]
    prev = None
    for t in times:
        if prev:
            print(f"{t}\t{(t-prev).total_seconds() // 3600}")
        else:
            print(t)
        prev = t
    print("")

### Show Interval Boundaries for Current Time for All Intervals ###

In [ ]:
t = HecTime.now()
t.midnight_as_2400 = False
print("Current Time\t\tIntvl\tTop of Prev\t\tTop of Next")
for intvl in Interval.getAll(lambda i: i.minutes > 0):
    prev = HecTime(t).adjustToIntervalOffset(intvl, 0)
    next = HecTime(prev).increment(1, intvl)
    print(f"{t}\t{intvl}\t{prev}\t{next}")

### Format Date With Style (Generate Date Style Table) ###

In [ ]:
width = 18
t = HecTime("02Jun1985, 12:34").atTimeZone("local")
html = '<html><pre><table border="1"><tr><th colspan="4" style="text-align:left;"><b>Base Styles</b></th></tr>'
for row in range(10):
    html += "<tr>"
    for col in (0, 10, 100, 110):
        style = row + col
        html += f"<td><b>{style}:</b> {t.date(style).rjust(width)}</td>"
    html += "</tr>"
html += '<tr><th colspan="4" style="text-align:left;"><b>Extended Styles</b></th></tr>'
for row in (-1, -2, -3):
    html += "<tr>"
    for col in (0, 10, 100, 110):
        style = row - col
        if style in (-3, -103, -113):
            html += "<td></td>"
        else:
            html += f"<td><b>{style}:</b> {t.date(style).rjust(width)}</td>"
    html += "</tr>"
html += "</table></pre></html>"
display_html(html, raw=True)  # type: ignore

### Format and Parse Date/Time Like `datetime.datetime` ###

In [ ]:
t = HecTime("02Jun1985, 12:34").atTimeZone("local")
print(f"HecTime object = {t}")
format = "%A, %B %d, %Y %H:%M:%S %z"
dateTimeStr = t.strftime(format)
print(f'Formatted using strftime("{format}")  : {dateTimeStr}')
t.setUndefined()
t.atTimeZone(None)
print(f"\nNew HecTime Object : {repr(t)}")
print(
    f'After parsing "{dateTimeStr}" with "{format}"  : {t.strptime(dateTimeStr, format)}'
)